# Data Science Essentials: Data Cleaning
    <Name>
    <Class>
    <Date>
    

In [14]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. Each test assigns 1 point to each question asked (so there are no non integer scores).
Using this dataset, answer the following questions.



1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.


In [2]:
def prob1():
    # read in data
    df = pd.read_csv('g_t_results.csv')
    # part 1
    col1 = df.isna().sum().idxmax()
    ans1 = (col1, df.isna().sum().max()/len(df[col1])*100)
    print("Answer 1: ", ans1)
    # part 2
    ans2 = ('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')
    print("Answer 2: ", ans2)
    # part 3
    grade3 = df.loc[df["Entering Grade Level"] == "3"]
    ans3 = len(grade3.loc[pd.to_numeric(grade3["OLSAT Verbal Score"]) > 30])
    print("Answer 3: ", ans3)
    # part 4
    ans4 = df.isna().sum().sum()
    print("Answer 4: ", ans4)

prob1()

Answer 1:  ('School Assigned', 75.21367521367522)
Answer 2:  ('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')
Answer 3:  1
Answer 4:  192


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [5]:
def prob2():
    # load in data
    df = pd.read_csv('imdb.csv')
    # part 1
    df = df.drop_duplicates(keep="first")
    print("Answer 1: ", df.shape)
    # part 2
    df = df.dropna()
    print("Answer 2: ", df.shape)
    # part 3
    df = df[df["duration"] > 0]     # remove movies that are zero minutes
    df = df[df["imdb_score"] < 10]  # remove movies w/ score out of range
    df = df[df["budget"] > 0]       # remove movies w/  zero budget
    df = df[df["gross"] > 0]        # remove movies that didn't gross
    ans3 = ['zero minutes or less,','movies with an IMDB score out of range,',
            'movies with zero budget,', 'and movies with zero gross rev.']
    print("Answer 3: I removed movies that were", ans3[0], ans3[1],'\n\t ',ans3[2], ans3[3])
    # part 4
    columns_2_drop = df.nunique() > 3
    cols = columns_2_drop.index[~columns_2_drop.values]
    df.drop(cols, axis=1, inplace=True)
    print(f"Answer 4: {tuple(cols)}" )
    # part 5
    df['movie_title'] = df['movie_title'].str.lower()
    print(f'Part 5:\n{df.head(5)}')
prob2()  

Answer 1:  (93, 13)
Answer 2:  (64, 13)
Answer 3: I removed movies that were zero minutes or less, movies with an IMDB score out of range, 
	  movies with zero budget, and movies with zero gross rev.
Answer 4: ('color', 'language')
Part 5:
       director_name  duration        gross  \
0    Martin Scorsese       240  116866727.0   
1        Shane Black       195  408992272.0   
2  Quentin Tarantino       187   54116191.0   
3   Kenneth Lonergan       186      46495.0   
4      Peter Jackson       186  258355354.0   

                                 genres                          movie_title  \
0          Biography|Comedy|Crime|Drama              the wolf of wall street   
1               Action|Adventure|Sci-Fi                           iron man 3   
2  Crime|Drama|Mystery|Thriller|Western                    the hateful eight   
3                                 Drama                             margaret   
4                     Adventure|Fantasy  the hobbit: the desolation of smaug 

### Problem 3
Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt for your convenience.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    
    2) Add two new features: 
		a) Remodeled: Whether or not a house has been remodeled with a Y if it has been
		   remodeled, or a N if it has not.
		
		b) TotalPorch: Using the 5 different porch/deck columns, create a new column that
		   provides the total square footage of all the decks and porches for each house.
    
	3) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    4) Add a constant column to the dataframe.

    5) Save a copy of the dataframe.

	6) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	7) Run an OLS using all numerical data regression on your model.  

	
Print the ten features that have the highest coef in your model. Then print the summary. Don't print the OLS

In [53]:
def prob3(missing='drop'):
    # load data
    df = pd.read_csv('housing.csv', index_col=0)

    missing = 'median'
    # 1. Identify and handle the missing data
    if missing == 'drop':
        missing_data = df.isna().sum()/len(df)
        mask = missing_data[missing_data > .95].index   # drop columns with over 95% of missing data
        df.drop(mask, axis=1, inplace=True)
    if missing == 'median':
        for col in df.select_dtypes(include=['number']).columns:    # fill numerical w/ median
            df[col].fillna(df[col].median(), inplace=True)
        for col in df.select_dtypes(include=['object']).columns:    # fill categorical w/ mode
            df[col].fillna(df[col].mode()[0], inplace=True)

    if missing == 'mean':
        for col in df.select_dtypes(include=['number']).columns:    # fill numerical w/ mean
            df[col].fillna(df[col].mean(), inplace=True)
        for col in df.select_dtypes(include=['object']).columns:    # fill categorical w/ mode
            df[col].fillna(df[col].mode()[0], inplace=True)

    # 2. Create two new features
    df['Remodeled'] = np.where(df['YearBuilt'] == df['YearRemodAdd'], 'N', 'Y') # 2.1 Remodeled

    df['TotalPorch'] = df[['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'WoodDeckSF']].sum(axis=1)   # 2.2 TotalPorch

    # 3. Identify var w/ nonnumerical vals that are misencoded
    df = pd.get_dummies(df, columns=['MSSubClass'], drop_first=True)    # MSSubClass = type of dwelling

    # 4. Add a constant column
    df['Constant'] = 1

    # 5. Save a copy of the dataframe
    df_copy = df.copy()

    # 6.1 Choose 4 categorical features important in predicting SalePrice
    important_categorical = ['BldgType', 'HouseStyle', 'Neighborhood', 'ExterQual']

    # 6.2 one-hot encode, & remove all other categorical features.
    df = pd.get_dummies(df, columns=important_categorical, drop_first=True)
    other_categorical = df.select_dtypes(include=['object']).columns
    df.drop(other_categorical, axis=1, inplace=True)

    # 7. Run an OLS (Ordinary Least Squares) regression on your model.
    X = df.drop('SalePrice', axis=1)
    y = df['SalePrice']
    results = sm.OLS(y, X).fit()

    high_coef_features = results.params.sort_values(ascending=False).head(10)
    results_as_html = results.summary().tables[1].as_html()
    result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]
    # Print
    print(high_coef_features)
    print(results.summary())
    return df_copy, result_df

df_copy, results_summary = prob3(missing='mean')
results_summary

Constant                1.092039e+06
Neighborhood_StoneBr    5.458710e+04
Neighborhood_NridgHt    4.197958e+04
Neighborhood_NoRidge    4.141052e+04
HouseStyle_1Story       3.024086e+04
HouseStyle_1.5Unf       2.522139e+04
HouseStyle_SFoyer       2.071130e+04
MSSubClass_75           1.992963e+04
HouseStyle_SLvl         1.969811e+04
GarageCars              1.395850e+04
dtype: float64
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.855
Method:                 Least Squares   F-statistic:                     103.0
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        08:26:06   Log-Likelihood:                -17094.
No. Observations:                1460   AIC:                         3.436e+04
Df Residuals:                    1375   BIC:                  

,coef,std err,t,P>|t|,[0.025,0.975]
LotFrontage,-120.0727,48.639,-2.469,0.014,-215.488,-24.657
LotArea,0.4072,0.097,4.190,0.000,0.217,0.598
OverallQual,11740.0000,1180.022,9.946,0.000,9421.907,14100.000
OverallCond,5322.1490,961.828,5.533,0.000,3435.339,7208.959
YearBuilt,296.7879,88.102,3.369,0.001,123.960,469.616
...,...,...,...,...,...,...
Neighborhood_Timber,-4466.7757,10100.000,-0.444,0.657,-24200.000,15300.000
Neighborhood_Veenker,12730.0000,12600.000,1.013,0.311,-11900.000,37400.000
ExterQual_Fa,-35650.0000,11500.000,-3.101,0.002,-58200.000,-13100.000
ExterQual_Gd,-40500.0000,5294.432,-7.650,0.000,-50900.000,-30100.000


### Problem 4

Using the copy of the dataframe you created in Problem 3, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [54]:
# one-hot encode all the categorical variables
all_categorical = df_copy.select_dtypes(include=['object']).columns
df_copy = pd.get_dummies(df_copy, columns=all_categorical, drop_first=True)

# Print database shape
print(df_copy.shape)

# Run OLS
X = df_copy.drop('SalePrice', axis=1)
y = df_copy['SalePrice']
results = sm.OLS(y, X).fit()

# Ten features with the highest coefficients
high_coef_features = results.params.sort_values(ascending=False).head(10)

# Model summary
results_as_html = results.summary().tables[1].as_html()
result_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

# Print
print(high_coef_features)
print(results.summary())
result_df

(1460, 262)
RoofMatl_Membran    759200.193637
RoofMatl_Metal      729030.866793
RoofMatl_WdShngl    718179.814752
RoofMatl_Tar&Grv    665923.458386
RoofMatl_CompShg    661380.394037
RoofMatl_WdShake    652750.689738
RoofMatl_Roll       651031.227479
GarageCond_Gd       133657.148872
GarageCond_Po       129471.440183
GarageCond_TA       128982.382236
dtype: float64
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     65.62
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        08:29:46   Log-Likelihood:                -16572.
No. Observations:                1460   AIC:                         3.366e+04
Df Residuals:                    1204   BIC:                         3.501e+04
D

,coef,std err,t,P>|t|,[0.025,0.975]
LotFrontage,51.7834,44.176,1.172,0.241,-34.887,138.454
LotArea,0.7212,0.110,6.564,0.000,0.506,0.937
OverallQual,6293.0086,1030.422,6.107,0.000,4271.386,8314.631
OverallCond,5664.4988,884.357,6.405,0.000,3929.446,7399.552
YearBuilt,353.0726,84.169,4.195,0.000,187.939,518.207
...,...,...,...,...,...,...
SaleCondition_Alloca,3099.1190,8888.512,0.349,0.727,-14300.000,20500.000
SaleCondition_Family,-164.4886,6104.952,-0.027,0.979,-12100.000,11800.000
SaleCondition_Normal,6039.7033,2912.233,2.074,0.038,326.087,11800.000
SaleCondition_Partial,-4258.3991,15000.000,-0.284,0.776,-33600.000,25100.000


The second model has a much higher R-squared value, which one would think means it's a significantly better model, however it's possible that this model is over fitting. The first model may be better, especially if it uses a combination of imperical knowledge (knowing what features are more known to influence sales price) with the data to make a model that combines both elements of modeling.